In [1]:
import importlib
import sys

# Google Colab 上で実行しているかどうかを判断するフラグ
ON_COLAB = "google.colab" in sys.modules
print("ON_COLAB:", ON_COLAB)

if ON_COLAB:
    # Google Drive にマウントする
    drive = importlib.import_module("google.colab.drive")
    drive.mount("/content/drive")

ON_COLAB: False


# Import

In [2]:
import numpy as np
# import polars as pl
import pandas as pd
from pathlib import Path
import matplotlib as plt
from pydantic import BaseModel, Field
# import dataclasses

# Config

In [6]:
SEED = 55
# ==========================
# Configクラスの定義
# ==========================
class Config(BaseModel):
    """プロジェクト全体の設定情報を保持するクラス
    Pydanticにより型保証とパス管理を行う。

    Args:
        BaseModel (_type_): データを読み込んでインスタンスを生成する
    """
    
    # 現在のnotebookファイルの２階層上のディレクトリを指定
    data_dir:Path = Field(
        Path(__file__).resolve().parents[2] / "data"
        if "__file__" in globals() else Path.cwd().parents[1] / "data",
        description = "データディレクトリへのパス"
    )
    
    train_file: str = Field("taxi_dataset.csv", description="学習データ")
    test_file: str = Field("taxi_dataset_for_upload.csv", description = "テスト用")
    random_seed: int = Field(SEED, description="乱数シード値")
    
    
    # === パス生成プロパティ ===
    @property
    def train_path(self) -> Path:
        return self.data_dir / self.train_file

    @property
    def test_path(self) -> Path:
        return self.data_dir / self.test_file

    # === データ読み込み関数 ===
    def load_train_data(self) -> pd.DataFrame:
        """学習用データをCSVから読み込む"""
        print(f"📂 Loading training data from: {self.train_path}")
        return pd.read_csv(self.train_path)

## EDA

In [7]:
# ==============================
# 🔍 EDA（探索的データ分析）関数
# ==============================
def quick_eda(df: pd.DataFrame, n_rows: int = 5) -> None:
    """
    データの基本的な構造と概要を表示する簡易EDA関数
    """
    print("===== 🧾 データの概要 =====")
    print(df.info())
    print("\n===== 🔢 数値統計量 =====")
    display(df.describe().T)
    print("\n===== 🧩 欠損値の数 =====")
    display(df.isnull().sum())
    print("\n===== 👀 先頭データ =====")
    display(df.head(n_rows))

## Mainスクリプト

In [8]:
# ==============================
# 🚀 メイン実行部
# ==============================

# --- Configの初期化 ---
cfg = Config()
print("✅ Config initialized successfully.")
print(f"データパス: {cfg.train_path}")

# --- データ読み込み ---
df = cfg.load_train_data()

# --- 簡易EDA ---
quick_eda(df)


✅ Config initialized successfully.
データパス: /app/data/taxi_dataset.csv
📂 Loading training data from: /app/data/taxi_dataset.csv
===== 🧾 データの概要 =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10950 entries, 0 to 10949
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   area      10950 non-null  int64 
 1   date      10950 non-null  object
 2   num_trip  10950 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 256.8+ KB
None

===== 🔢 数値統計量 =====


,count,mean,std,min,25%,50%,75%,max
area,10950.0,5.500000,2.872412,1.0,3.0,5.5,8.0,10.0
num_trip,10950.0,4896.238356,6247.476719,76.0,891.0,1785.5,6012.5,38639.0



===== 🧩 欠損値の数 =====


area        0
date        0
num_trip    0
dtype: int64


===== 👀 先頭データ =====


,area,date,num_trip
0,1,2017-01-01,1275
1,1,2017-01-02,623
2,1,2017-01-03,692
3,1,2017-01-04,794
4,1,2017-01-05,827
